### Heroes Of Pymoli Data Analysis


In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os, sys
# File to Load (Remember to Change These)
path = "pythonmmo/Resources/data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(path)
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


'/home/jurioste/github/jurioste4.github.io'

In [4]:

## total player count 
total = purchase_data['SN'].count()
gender_total = purchase_data.groupby("Gender")
print("Total number of Players",total)

Total number of Players 780


In [3]:
# for name, group in gender_total:
#     print (name)
#     print (group)


In [5]:
# get player count , total sales and player percentage all by Gender 
total_gender= gender_total.nunique( )["SN"]
total_money = gender_total["Price"].sum()

# Player percentage by gender then change date type. 
player_percentage = total_gender / total*100
PC_player = player_percentage.astype(int)


In [6]:
# DF from Percentage and total money 
df2 = pd.DataFrame(dict(player_percentage = PC_player, total_money = total_money), index=PC_player.index)
# Convert total_gender out put to data frame
df = total_gender.to_frame().reset_index()



In [7]:
# join df and df2 to one main data frame to use for graphs in D3 

main_df = pd.merge(df,df2, how='inner', on='Gender',
                      suffixes=('_m','_n'), indicator=True)
main_df.drop(['_merge'], axis=1, inplace=True)
main_df.columns = [ 'Gender' , 'player_count' , 'PC','revenue']
# data frame to CSV 
# main_df.to_csv(r"Resources/gender.csv", encoding='utf-8' , index = None, header=True)
main_df

,Gender,player_count,PC,revenue
0,Female,81,10,361.94
1,Male,484,62,1967.64
2,Other / Non-Disclosed,11,1,50.19


In [8]:
gender_total = purchase_data.groupby("Gender")["SN"].nunique() / total * 100

gender_total


# pie Graph here

Gender
Female                   10.384615
Male                     62.051282
Other / Non-Disclosed     1.410256
Name: SN, dtype: float64

In [9]:


age_df = purchase_data

## Age Demographics and  Analysis (Age)

In [10]:
## create bin for age and group names

Age_bin = [0,10,14,19,24,29,34,41,100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# create new colum for df then add the bin with labes
age_df["age group"] = pd.cut(purchase_data["Age"],Age_bin,labels=group_names )

# set groupby new colum 
age_costgroup = age_df.groupby("age group")





In [11]:

# Get total user count by age , sales and percentage of users all by age 
age_count = age_costgroup.nunique()["SN"]
age_data = age_count.to_frame().reset_index()

Percentage = (age_count/total) * 100
pc_age =  Percentage.to_frame().reset_index()

# Crate ate new df and lable any dulcates 
group_one = pd.merge(age_data,pc_age, how='inner', on='age group',
                      suffixes=('_m','_n'), indicator=True)
group_one.drop(['_merge'], axis=1, inplace=True)

g_one = group_one.rename({'age group':'Age_Group','SN_m':'User_total','SN_n':'Player_PC'},axis=1)
g_one

,Age_Group,User_total,Player_PC
0,<10,24,3.076923
1,10-14,15,1.923077
2,15-19,107,13.717949
3,20-24,258,33.076923
4,25-29,77,9.871795
5,30-34,52,6.666667
6,35-39,38,4.871795
7,40+,5,0.641026


In [12]:
## same thing as cell above. 

age_spent = age_costgroup["Price"].sum()
price_age = age_spent.to_frame().reset_index()

avg_price = age_costgroup["Price"].mean()
price_avg = avg_price.to_frame().reset_index()

group_two = pd.merge(price_age,price_avg, how='inner', on='age group',
                      suffixes=('_m','_n'), indicator=True)

group_two.drop(['_merge'], axis=1, inplace=True)
g_two = group_two.rename({'age group':'Age_Group','Price_m':'Total_Revenue','Price_n':'Mean_Price'},axis=1)
g_two

,Age_Group,Total_Revenue,Mean_Price
0,<10,108.96,3.405000
1,10-14,50.95,2.681579
2,15-19,412.89,3.035956
3,20-24,1114.06,3.052219
4,25-29,293.00,2.900990
5,30-34,214.00,2.931507
6,35-39,170.92,3.488163
7,40+,14.99,2.998000


In [14]:
## now merage both Dataframes in to one all by age. 

df_age = pd.merge(g_one,g_two, how='inner', on='Age_Group',
                    suffixes=('_m','_n'), indicator=True)
df_age.drop(['_merge'], axis=1, inplace=True)

# df_age.to_csv(r"pythonmmo/Resources/ageplot.csv", encoding='utf-8' , index = None, header=True)
df_age

,Age_Group,User_total,Player_PC,Total_Revenue,Mean_Price
0,<10,24,3.076923,108.96,3.405000
1,10-14,15,1.923077,50.95,2.681579
2,15-19,107,13.717949,412.89,3.035956
3,20-24,258,33.076923,1114.06,3.052219
4,25-29,77,9.871795,293.00,2.900990
5,30-34,52,6.666667,214.00,2.931507
6,35-39,38,4.871795,170.92,3.488163
7,40+,5,0.641026,14.99,2.998000


In [19]:
bar_df =pd.DataFrame([df_age.Age_Group,df_age.User_total,df_age.Total_Revenue]).transpose()
# bar_df.to_csv(r"pythonmmo/Resources/bar_df.csv", encoding='utf-8' , index = None, header=True)
bar_df

,Age_Group,User_total,Total_Revenue
0,<10,24,108.96
1,10-14,15,50.95
2,15-19,107,412.89
3,20-24,258,1114.06
4,25-29,77,293
5,30-34,52,214
6,35-39,38,170.92
7,40+,5,14.99


* taken the information above to display based on total 




## Top Spenders

In [72]:
#group the data frame purchase_data and group by the SN colum
top_spender =purchase_data.groupby("SN")
# then do a count,mean and sum of colums below .. 
spender_count = top_spender["Purchase ID"].count()
avg_spender = top_spender["Price"].mean()

spender_total = top_spender["Price"].sum()
#place the outputs of the three colums in to a data frame 
Most_spenders_pd = pd.DataFrame({"Count": spender_count,
                                 "Avg spent": avg_spender,
                                 "Spender_total": spender_total})



#now I have top spenders with total itams payed for and total spent 
#this is intresting as some spent more but bout less items 
Most_spenders_pd.sort_values(by='Spender_total',ascending=False).head(10)

,Count,Avg spent,Spender_total
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10
Ilarin91,3,4.233333,12.70
Ialallo29,3,3.946667,11.84
Tyidaim51,3,3.943333,11.83
Lassilsala30,3,3.836667,11.51


## Most Popular Items

In [78]:
most = purchase_data[["Item ID", "Item Name", "Price"]]
item_grup = most.groupby(["Item ID", "Item Name"])  

item_count = item_grup["Price"].count()

Item_price = item_grup["Price"].sum()

real_cost = Item_price/item_count


Most_popular_Stuff = pd.DataFrame({"total count sold": item_count,
                                  "Total Sold": Item_price,
                                  "Price per item": real_cost})

Most_popular_Stuff.sort_values(by='Total Sold',ascending=False).head(10)

,,total count sold,Total Sold,Price per item
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.23
82,Nirvana,9,44.10,4.90
145,Fiery Glass Crusader,9,41.22,4.58
92,Final Critic,8,39.04,4.88
103,Singed Scalpel,8,34.80,4.35
59,"Lightning, Etcher of the King",8,33.84,4.23
108,"Extraction, Quickblade Of Trembling Hands",9,31.77,3.53
78,"Glimmer, Ender of the Moon",7,30.80,4.40
72,Winter's Bite,8,30.16,3.77
